In [1]:
# Install Java, Spark, and Findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.7.tgz
!tar xf spark-2.4.6-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.6-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()



In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-08-06 15:00:16--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.70MB/s    in 0.2s    

2020-08-06 15:00:17 (4.70 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [6]:
from pyspark.sql.functions import lit
#from pyspark import SparkFiles

#url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Gift_Card_v1_00.tsv.gz"
#spark.sparkContext.addFile(url)

#sc = spark.sparkContext

#sc._jsc.hadoopConfiguration().set("fs.s3a.access.key", "ACCESSKEY")
#sc._jsc.hadoopConfiguration().set("fs.s3a.secret.key", "SecretKey")
#sc._jsc.hadoopConfiguration().set("fs.s3a.impl","org.apache.hadoop.fs.s3a.S3AFileSystem")
#sc._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "s3.amazonaws.com")

# s3a://amazon-reviews-pds/tsv/amazon_reviews_us_Sports_v1_00.tsv.gz

giftcard_df = spark.read.option("header", "true").csv("gdrive/My Drive/amazon_reviews_us_Gift_Card_v1_00.tsv.gz", inferSchema=True, sep="\t")

#giftcard_df = spark.read.format('csv').options(header='true', inferSchema='true', delimiter='\t').load('s3a://amazon-reviews-pds/tsv/amazon_reviews_us_Gift_Card_v1_00.tsv.gz')

giftcard_df = giftcard_df.withColumn('category',lit('GiftCard'))

giftcard_df.show(truncate=False)

+-----------+-----------+--------------+----------+--------------+----------------------------------------------------------------------------+----------------+-----------+-------------+-----------+----+-----------------+----------------------------------------------------+-------------------------------------------------------------------------------------------------------+-------------------+--------+
|marketplace|customer_id|review_id     |product_id|product_parent|product_title                                                               |product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|review_headline                                     |review_body                                                                                            |review_date        |category|
+-----------+-----------+--------------+----------+--------------+----------------------------------------------------------------------------+----------------+-----------+------------

In [7]:
giftcard_df.count()

149086

In [8]:
customer_df = giftcard_df.select(["customer_id","category","review_id"])\
  .groupby("customer_id","category")\
  .agg({"review_id":"count"})

customer_df = customer_df.withColumnRenamed("count(review_id)","customer_count")
customer_df.show(truncate=False)

+-----------+--------+--------------+
|customer_id|category|customer_count|
+-----------+--------+--------------+
|15718513   |GiftCard|1             |
|37908228   |GiftCard|1             |
|51029369   |GiftCard|1             |
|1088864    |GiftCard|1             |
|50525748   |GiftCard|1             |
|2173713    |GiftCard|1             |
|4053665    |GiftCard|1             |
|23986344   |GiftCard|1             |
|275361     |GiftCard|1             |
|1188277    |GiftCard|1             |
|47907308   |GiftCard|1             |
|50296057   |GiftCard|1             |
|1316801    |GiftCard|1             |
|388656     |GiftCard|1             |
|20373502   |GiftCard|1             |
|26913738   |GiftCard|1             |
|37125972   |GiftCard|1             |
|41885922   |GiftCard|1             |
|37778850   |GiftCard|1             |
|3093324    |GiftCard|1             |
+-----------+--------+--------------+
only showing top 20 rows



In [9]:
product_df = giftcard_df.select('product_id','product_title').distinct()
product_df.show(truncate=False)

+----------+--------------------------------------------------------------------------------------------+
|product_id|product_title                                                                               |
+----------+--------------------------------------------------------------------------------------------+
|B00CHQKNUM|Amazon.com Gift Card in a Black Gift Box (Previous Generation Card Designs)                 |
|B00CRQ4ZOM|Amazon Gift Card - Print - #1 Dad                                                           |
|B00H5BMZ32|Amazon eGift Card - Eyes Know What You Did (Animated) [Hallmark]                            |
|BT00CTP84S|Amazon.com Gift Card in a Greeting Card (Various Designs)                                   |
|B004LLILN2|Amazon eGift Card - Happy Mother's Day (Child Art)                                          |
|B00N59437O|Amazon Gift Card - Email - Birthday Birthstone: January (Garnet)                            |
|B005J0444E|Amazon.com Gift Card in a Greeting

In [10]:
from pyspark.sql.types import DateType

review_df = giftcard_df.select('review_id','customer_id','product_id','product_parent','category','review_date')
review_df=review_df.withColumn('review_date',review_df['review_date'].cast(DateType()))

review_df.show()

+--------------+-----------+----------+--------------+--------+-----------+
|     review_id|customer_id|product_id|product_parent|category|review_date|
+--------------+-----------+----------+--------------+--------+-----------+
|R27ZP1F1CD0C3Y|   24371595|B004LLIL5A|     346014806|GiftCard| 2015-08-31|
| RJ7RSBCHUDNNE|   42489718|B004LLIKVU|     473048287|GiftCard| 2015-08-31|
|R1HVYBSKLQJI5S|     861463|B00IX1I3G6|     926539283|GiftCard| 2015-08-31|
|R2HAXF0IIYQBIR|   25283295|B00IX1I3G6|     926539283|GiftCard| 2015-08-31|
| RNYLPX611NB7Q|     397970|B005ESMGV4|     379368939|GiftCard| 2015-08-31|
|R3ALA9XXMBEDZR|   18513645|B004KNWWU4|     326384774|GiftCard| 2015-08-31|
|R3R8PHAVJFTPDF|   22484620|B004LLIKVU|     473048287|GiftCard| 2015-08-31|
|R18WWEK8OIXE30|   14765851|BT00CTP2EE|     775486538|GiftCard| 2015-08-31|
|R1EGUNQON2J277|   18751931|B004LLIKVU|     473048287|GiftCard| 2015-08-31|
|R21Z4M4L98CPU2|   15100528|B004W8D102|     595099956|GiftCard| 2015-08-31|
| R6JH7A117F

In [11]:
vine_df = giftcard_df.select('review_id','star_rating','helpful_votes','total_votes','vine')
vine_df.show()

 

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R27ZP1F1CD0C3Y|          5|            0|          0|   N|
| RJ7RSBCHUDNNE|          5|            0|          0|   N|
|R1HVYBSKLQJI5S|          5|            0|          0|   N|
|R2HAXF0IIYQBIR|          1|            0|          0|   N|
| RNYLPX611NB7Q|          5|            0|          0|   N|
|R3ALA9XXMBEDZR|          5|            0|          0|   N|
|R3R8PHAVJFTPDF|          5|            0|          0|   N|
|R18WWEK8OIXE30|          5|            0|          0|   N|
|R1EGUNQON2J277|          1|            0|          0|   N|
|R21Z4M4L98CPU2|          5|            0|          0|   N|
| R6JH7A117FHFA|          5|            0|          0|   N|
|R1XZHS8M1GCGI7|          5|            1|          1|   N|
|R1DAI0N03SKRJN|          5|            1|          1|   N|
|R2F6SKZOEYQRU3|          5|            

## Write DataFrame to RDS

In [12]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://database-1.chwtrhvyl69j.us-east-2.rds.amazonaws.com/BigData"
config = {"user":"root", 
          "password": "Password", 
          "driver":"org.postgresql.Driver"}



In [13]:
customer_df.write.jdbc(url=jdbc_url, table="customers", mode=mode, properties=config)

In [14]:
product_df.write.jdbc(url=jdbc_url, table="products", mode=mode, properties=config)

In [15]:
review_df.write.jdbc(url=jdbc_url, table="review_id_table", mode=mode, properties=config)

In [16]:
vine_df.write.jdbc(url=jdbc_url, table="vine_table", mode=mode, properties=config)